In [ ]:
#| export
from __future__ import annotations
# TODO: check what happens if we uncoment above line

In [ ]:
#| default_exp tree_widget

In [ ]:
# %reload_ext autoreload
# %autoreload 0

# Tree Widget
> Simple widgets composition to browse object trees

## Prologue

In [ ]:
#| export
import asyncio
from pathlib import Path
from typing import cast
from typing import ClassVar
from typing import Collection
from typing import Generator
from typing import Iterable
from typing import NamedTuple
from typing import Mapping
from typing import Protocol
from typing import Type
from typing import TypeAlias

import ipywidgets as W
import traitlets as T
from ipytree import Node
from ipytree import Tree
from traitlets import Instance
from traitlets import List
from traitlets import observe

from vutil.vario import is_listy
from vwidget.base_widget import SelectDetailApp


In [ ]:
import dataclasses
import fastcore.all as F
# import ipyevents as EVT
from fastcore.test import *  # type: ignore [reportWildcardImportFromLibrary]

from vutil.imports import AD
from vutil.imports import setup_console
from vutil.logger_loguru import config_logger
from vutil.test import test_raises
from vutil.test import test_is_not
from vwidget.base_widget import DebugWidget
from vwidget.base_widget import ThemerWidget


 ----

In [ ]:
pre = '/Users/vic/dev/repo/project/TBOs/proj/tbos/'

In [ ]:
# %%html
# <style>
#     .cell-output-ipywidget-background {background-color: rgba(0, 0, 0, 0.0) !important;}
#     .jstree {background-color: var(--vscode-editor-background) !important;)}
#     .jstree-anchor {color: var(--vscode-editor-foreground) !important;)}
# </style>


 ----

## AnyTreeWidget

In [ ]:
#| exporti

def _get_name(name: str | None, value) -> str:
    try:
        return name or getattr(value, 'name')
    except AttributeError:
        pass
    return type(value).__name__ if is_listy(value) else str(value)


In [ ]:
#| export
class NamedP(Protocol):
    @property
    def name(self) -> str: ...

AnyTreePath: TypeAlias = tuple[int, ...]

class AnyNodeItem(NamedTuple):
    path: AnyTreePath
    node: AnyTreeNode

class ValueNode(Node):
    value: object
    # value = T.Instance(klass=object, read_only=True)
    _icon_: ClassVar[str] = 'archive'
    def __init__(self, value:object, name: str | None = None, **kwargs):
        self.value = value
        # self.set_trait('value', value)
        super().__init__(
            _get_name(name, value), 
            icon=kwargs.pop('icon', self._icon_), 
            **kwargs)
    def _repr_keys(self):
        return ('value', 'name', 'selected')
    def __getitem__(self, path):
        raise IndexError(f'ValueNode has no children: {self.name}')
    def _iter(self, at: AnyTreePath | None = None) -> Generator[tuple[AnyTreePath, AnyTreeNode], None, None]:
        yield (at or (), self)

class FolderNode(Node):
    value: Collection[object]
    # value = T.Tuple(read_only=True).tag(trait=T.Instance(object))
    _icon_closed_: ClassVar[str] = 'folder'
    _icon_opened_: ClassVar[str] = 'folder-open'
    _close_icon_: ClassVar[str] = 'chevron-down'
    _open_icon_: ClassVar[str] = 'chevron-right'

    def __init__(self, 
            value: Collection[object], 
            name: str | None = None, 
            nodes: Collection[Node] = (), 
            **kwargs
        ):
        self._to = None
        self.value = value
        # self.set_trait('value', value)
        opened = kwargs.pop('opened', False)
        super().__init__(
            name = _get_name(name, value), 
            nodes = nodes,
            icon = kwargs.pop('icon', self._icon_opened_ if opened else self._icon_closed_), 
            opened = opened,
            close_icon = kwargs.pop('close_icon', self._close_icon_),
            open_icon = kwargs.pop('open_icon', self._open_icon_),
            **kwargs)
    def _repr_keys(self):
        return ('name', 'selected', 'opened',)
    def _iter(self, at: AnyTreePath | None = None) -> Generator[AnyNodeItem, None, None]:
        at = at or ()
        yield AnyNodeItem(at, self)
        for i,node in enumerate(self.nodes):  # type: ignore
            yield from node._iter(at + (i,))
    def __getitem__(self, path: int | AnyTreePath) -> ValueNode | FolderNode:
        path = (path, ) if isinstance(path, int) else path
        node = self.nodes[path[0]]  # type: ignore
        return node if len(path) == 1 else node[path[1:]]

class TreeAdapter:
    _items_str_: ClassVar[str] = 'items'

    @classmethod
    def is_folder(cls, x: object) -> bool:
        if isinstance(x, Collection) and not isinstance(x, (str, bytes)):
            return True
        items = getattr(x, cls._items_str_, None)
        return bool(items and isinstance(items, Iterable))

    @classmethod
    def items(cls, value) -> Generator[tuple[str, object], None, None]:
        if isinstance(value, Mapping):
            for kv in value.items():
                yield kv
        elif isinstance(value, Iterable):
            for v in value:
                yield (_get_name(None, v), v)
        elif hasattr(value, cls._items_str_):
            pp = [(_get_name(None, v), v) for v in value.items]
            pp = sorted(pp, key=lambda x: x[0])
            for p in pp:
                yield p
        else:
            raise TypeError(f"{value = }")

AnyTreeNode: TypeAlias = ValueNode | FolderNode

_FOLDER_SENTINEL = ValueNode(None, name='FOLDER_SENTINEL')

class AnyTreeWidget(Tree):
    source = List(())
    _adapter_: ClassVar[TreeAdapter] = TreeAdapter()
    _value_node_: ClassVar[Type[ValueNode]] = ValueNode
    _folder_node_: ClassVar[Type[FolderNode]] = FolderNode
    
    value = T.Tuple().tag(trait=Instance(Node))
    # selected_nodes = Tuple(read_only=True).tag(trait=Instance(Node),sync=True, **widget_serialization)
    selected_paths = T.Tuple().tag(trait=Instance(tuple))

    def __init__(self, 
            source: Collection[object], 
            adapter: Type[TreeAdapter] | None = None, 
            select: AnyTreePath = (),
            expand: AnyTreePath = (),
            **kwargs):
        if adapter:
            setattr(self, '_adapter_', adapter())
        super().__init__(
            stripes=kwargs.pop('stripes', True), 
            multiple_selection=kwargs.pop('multiple_selection', False), 
            animation=100, 
            **kwargs)
        self.layout.overflow = 'scroll scroll'  # type: ignore
        self.source = source
        self._link = W.dlink((self, 'selected_nodes'), (self, 'value'))
        if select:
            self.select(select)
        if expand:
            self.expand(expand)

    def _setup_node(self, value, name: str | None = None, **kwargs):
        adapter = self._adapter_
        if adapter.is_folder(value):
            nodes = [self._setup_node(v, name=k) for k,v in adapter.items(value)]
            # nodes = [_FOLDER_SENTINEL]
            folder = self._folder_node_(value, name=_get_name(name, value), nodes=nodes, **kwargs)
            folder.observe(self.handle_click, 'opened')  # type: ignore
            return folder
        else:
            return self._value_node_(value, name=name)  
    
    # ---------- traversal
    def _iter(self, at: AnyTreePath | None = None) -> Generator[AnyNodeItem, None, None]:
        for i, node in enumerate(cast(Collection, self.nodes)):
            yield from node._iter((at or ())+(i,))
    
    def find_node(self, value: object) -> AnyTreeNode | None:
        for path, node in self._iter():
            if node.value == value:
                return node
        return None
    def find_all_nodes(self, value: object) -> tuple[AnyTreeNode, ...]:
        return tuple(node for path, node in self._iter() if node.value == value)
    def paths_of_value(self, value: object) -> tuple[AnyTreePath, ...]:
        return tuple(path for path, _ in self._iter() if _.value == value)
    def nodes_of_value(self, value: object) -> tuple[AnyTreeNode, ...]:
        return tuple(node for _, node in self._iter() if node.value == value)
    def path_of_node(self, node: AnyTreeNode) -> AnyTreePath | None:
        for path,_ in self._iter():
            if _ is node:
                return path
        return None
    
    def __getitem__(self, path: int | AnyTreePath) -> AnyTreeNode:
        _path: AnyTreePath = (path, ) if isinstance(path, int) else path
        node = self.nodes[_path[0]]
        return node if len(_path) == 1 else node[_path[1:]]
    
    # ---------- UX
    def handle_click(self, event):
        folder = event['owner']
        # if folder.nodes[0] is _FOLDER_SENTINEL:
        #     adapter = self._adapter_
        #     nodes = [self._setup_node(v, name=k) for k,v in adapter.items(folder.value)]
        #     folder.nodes = nodes
        #     # with self.hold_trait_notifications():
        #     #     # folder.remove_node(_FOLDER_SENTINEL)
        #     #     folder.add_node(nodes[0])
        #     #     folder.add_node(nodes[1])
        #     # loop = asyncio.get_event_loop()
        #     # loop.call_soon(lambda: self.handle_click(event))
        #     return
        folder.icon = folder._icon_opened_ if event['new'] else folder._icon_closed_

    def select(self, path_or_node: AnyTreePath | AnyTreeNode | object, scroll: bool = False):
        if isinstance(path_or_node, tuple):
            nodes = (self[path_or_node], )
        elif isinstance(path_or_node, (ValueNode, FolderNode)):
            nodes = (path_or_node, )
        else:
            nodes = self.nodes_of_value(path_or_node)
        self.value = nodes

    def expand(self, path_or_node: AnyTreePath | AnyTreeNode | object, scroll: bool = False):
        if isinstance(path_or_node, tuple):
            path = path_or_node
        elif isinstance(path_or_node, (ValueNode, FolderNode)):
            path = self.path_of_node(path_or_node)
        else:
            path = ()
            paths = self.paths_of_value(path_or_node)
            if paths:
                path = paths[0]
        if path:
            node = self
            for _ in path:
                node = node[_]
                if not node.opened:
                    node.opened = True
            # self[path].opened = True
        # if scroll:
        #     self.scroll_to_node(node)
    
    @observe('source')
    def _observe_source(self, change):
        source: Collection[object] = change['new']
        nodes, open_folder = [], False
        for v in source:
            n = self._setup_node(v, name=None, opened=open_folder)
            if open_folder and isinstance(n, self._folder_node_):
                open_folder = False
            nodes.append(n)
        self.nodes = nodes

    @observe('value')
    def _observe_value(self, change):
        new_selected = change['new']
        # print(f"{change['old'] = }, {change['new'] = }")
        if new_selected != self.selected_nodes:
            # this code path is taken only when setting the value programmatically
            with self.hold_trait_notifications():
                for node in change['new']:
                    if not node.selected:
                        node.selected = True
                selected = change['old']
                if selected is not T.Undefined:
                    for node in selected:
                        if node.selected:
                            node.selected = False
        # print(f"{self.value =}")
        # # self.notify_change = lambda x: None  # type:ignore[assignment]
        # # try:
        # #     self.selected_paths = tuple(self.path_of_node(_) for _ in self.value)
        # # finally:
        # #     del self.notify_change
        # self.set_trait('selected_nodes', tuple(self.path_of_node(_) for _ in self.value))
        self.selected_paths = tuple(self.path_of_node(_) for _ in self.value)  # type:ignore

    @observe('selected_paths')
    def _observe_selected_paths(self, change):
        new_value = tuple(self[_] for _ in change['new'])
        if new_value != self.value:
            self.value = tuple(self[_] for _ in change['new'])
        

    # @observe('selected_nodes')
    # def _observe_selected_nodes(self, change):
    #     v = change['new']
    #     self.value = tuple(node.value for node in v)
    #     # print(f"value = {self.value}")

    # @property
    # def value(self) -> object | tuple[object, ...] | None: return self._value
    # @value.setter
    # def value(self, values: tuple[object, ...] | None):
    #     vv: tuple[object, ...] = values or ()
    #     if not self.multiple_selection:
    #         vv = vv[:1]
    #     to_select = [self.find(v) for v in vv]
    #     for _ in cast(Collection, self.selected_nodes):
    #         _.selected = False
    #     for _ in to_select:
    #         if _:
    #             _.selected = True
    #     self._value = vv
    #     # print(f"{self._value = }")


In [ ]:
node = ValueNode(7, 'node')
test_eq(f"{node}", "ValueNode(value=7, name='node', selected=False)")

folder = FolderNode([7, 13], 'folder')
test_eq(f"{folder}", "FolderNode(name='folder', selected=False, opened=False)")
test_eq(folder.value, [7, 13])


In [ ]:
data = []
tree = AnyTreeWidget(data)
tree


AnyTreeWidget(animation=100, layout=Layout(overflow='scroll scroll'), multiple_selection=False)

In [ ]:
test_eq(tree.value, ())
test_eq(tree.selected_nodes, ())
test_eq(tuple(_ for _ in tree._iter()), ())


In [ ]:
data = [1, 2]
tree = AnyTreeWidget(data)
tree


AnyTreeWidget(animation=100, layout=Layout(overflow='scroll scroll'), multiple_selection=False, nodes=(ValueNo…

In [ ]:
test_eq(tree.value, ())
test_eq(tree.selected_nodes, ())
test_eq(tuple(_[1].value for _ in tree._iter()), (1, 2))


In [ ]:
data = [1, [2, 3], 4, 5, [6, 7, [8, 9], 10]]
tree = AnyTreeWidget(data)
tree

AnyTreeWidget(animation=100, layout=Layout(overflow='scroll scroll'), multiple_selection=False, nodes=(ValueNo…

In [ ]:
tree.expand((4, 2))
test_eq(tree[4].opened, True)
test_eq(tree[4, 2].opened, True)
test_eq(tree.value, ())


In [ ]:
tree.select((4, 2))
test_eq(tree[4, 2].selected, True)
test_eq(tree.selected_nodes, ())


In [ ]:
tree.value = ()
test_eq(tuple(_[1].value for _ in tree._iter()), 
        (1, [2, 3], 2, 3, 4, 5, [6, 7, [8, 9], 10], 6, 7, [8, 9], 8, 9, 10))
tree.nodes[1][1].selected = True
test_eq(tree.value, ())


We've got to wait for the round-trip between  server and client to get the updated widgets values.

In [ ]:
test_eq(tree.value, (tree[1, 1],))
test_eq(tree.value, tree.selected_nodes)
test_eq(tree.selected_paths, ((1, 1),))


In [ ]:
tree.value = ()
test_eq(tree.value, ())
test_ne(tree.value, tree.selected_nodes)  # must wait traitlets sync


In [ ]:
test_eq(tree.value, tree.selected_nodes)


In [ ]:
tree.selected_paths = ((1, 0), )
test_eq(tree.value, (tree[1, 0],))
test_ne(tree.value, tree.selected_nodes)


In [ ]:
test_eq(tree.value, tree.selected_nodes)


In [ ]:
data = [1, [2, 3], 4, 5, [6, 7, 8, 9], 10, [11, [12, [13, 14, 15]]], 6]
tree = AnyTreeWidget(data)
tree


AnyTreeWidget(animation=100, layout=Layout(overflow='scroll scroll'), multiple_selection=False, nodes=(ValueNo…

In [ ]:
n = tree.find_node(4)
test_is_not(n, None)
test_eq((n.value, n.name), (4, '4'))  # type: ignore

nn = tree.find_all_nodes(6)
test_eq(len(nn), 2)
test_eq((nn[0].value, nn[0].name), (6, '6'))
test_eq((nn[1].value, nn[1].name), (6, '6'))


In [ ]:
with test_raises(IndexError):
    tree[1, 0, 0]
with test_raises(IndexError):
    tree[(12,)]
with test_raises(IndexError):
    tree[()]
test_eq(tree[1, 0].value, 2)
test_eq(tree[1, 0].value, 2)
test_eq(tree[6, 1, 1, 0].value, 13)
test_eq(tree[6, 1, 1, -1].value, 15)
test_eq(tree[6, 1, 1].value, [13, 14, 15])


In [ ]:
test_eq(tree.paths_of_value(1), ((0,), ))
test_eq(tree.paths_of_value(6), ((4, 0, ), (7,), ))
test_eq(tree.path_of_node(tree[4, 0]), (4, 0, ))
test_eq(tree.paths_of_value(13), ((6, 1, 1, 0), ))
test_eq(tree.path_of_node(tree[6, 1, 1, 0]), (6, 1, 1, 0))


In [ ]:
# tree.unobserve_all()


In [ ]:
dcons = DebugWidget(show=False)
tree.observe(
    lambda x: dcons.log(str(tree.selected_paths)), names='selected_paths')  # type: ignore

app = SelectDetailApp(tree, dcons=dcons)


Box(children=(SelectDetailApp_CSS(), ReflectWidget(), SelectDetailApp(children=(AnyTreeWidget(animation=100, l…

DebugWidget(children=(Box(children=(Button(description='clear', layout=Layout(width='2em'), style=ButtonStyle(…

In [ ]:
tw = ThemerWidget()
tw

ThemerWidget()

## TreeSelect

In [ ]:
#| export
class TreeSelect(SelectDetailApp):
    _css = """
        .widget-output .jp-OutputArea {
            background-color: transparent;
            color: var(--vscode-editor-foreground);
        }
        .jstree {background-color: var(--vscode-editor-background) !important;)}
        .jstree-anchor {color: var(--vscode-editor-foreground) !important;)}
        a.jstree-anchor { pointer-events: none; }  # disable click, vscode bug
    """
    def _on_select_change(self, changed):
        nodes = tuple(AnyNodeItem(self.tree.path_of_node(_), _) for _ in changed['new'])  # type: ignore
        self.value = dict(select=nodes)
        self.d()

    # def setup_ux(self, state):
    #     super().setup_ux(state)
    #     tree = self.select
    #     tree.observe(lambda x: self._on_select_change(x), names='selected_nodes')  # type: ignore

        # super()._setup_ux()
        # dcons = self.dcons
        # if dcons:
        #     tree.observe(lambda x: dcons.log(str(tree.value)), names='selected_nodes')  # type: ignore
    
    def __init__(self, 
            data, 
            tree_class: Type[AnyTreeWidget] = AnyTreeWidget, 
            multiple_selection = False,
            expand: AnyTreePath = (),
            select_path: AnyTreePath = (),
            **kwargs
        ):
        self.tree = tree_class(data, multiple_selection=multiple_selection, expand=expand)
        super().__init__(self.tree, **kwargs)
        if select_path:
            loop = asyncio.get_event_loop()
            loop.call_later(0.5, lambda: self.tree.select(select_path))


In [ ]:
data = [1, [2, 3], 4, 5, [6, 7, 8, 9], 10, [11, [12, [13, 14, 15]]], 6]
# tree = AnyTreeWidget(data)

# height = 220
# dcons = DebugWidget(show=False, height=height-4)
# dcons = DebugWidget(show=False)
# tree.observe(lambda x: dcons.log(str(tree.value)), names='selected_nodes')  # type: ignore

# select_detail_app(tree, dcons.form, height=height)
app = TreeSelect(data, multiple_selection=True, pane_widths=['120px', 4, 4])


Box(children=(SelectDetailApp_CSS(), ReflectWidget(), TreeSelect(children=(AnyTreeWidget(animation=100, layout…

Node interactive modifications are reflected asynchronously.

In [ ]:
old_value = app.value
for _ in app.tree.selected_nodes:  # type: ignore
    _.selected = False
app.tree[1, 0].selected = True # or app.tree.value = (app.tree[1, 0], )
test_eq(app.value, old_value)


Unlike setting the widgets value directlly

In [ ]:
app.tree.value = (app.tree[1, 1], )
test_eq(app.value, {'select': (app.tree[1, 1], )})
test_eq(app.tree.selected_nodes, (app.tree[1, 0], ))


but note app.value is updated asynchonously in a notebook context

In [ ]:
test_eq(app.value, {'select': (app.tree[1, 1],)})
test_eq(app.tree.value, app.tree.selected_nodes)
test_eq(app.tree.selected_nodes, (app.tree[1, 1], ))


In [ ]:
n = app.tree.find_node(4)
test_eq(n, app.tree[2])

In [ ]:
data = [{'a':1, 'b':[2, 3], 'c':4, 'd':5, 'e':[6, 7, 8, 9], 'f':10}]
app.tree.source = data


In [ ]:
n = app.tree.find_node(4)
assert n is not None
test_eq(n.name, 'c')

In [ ]:
@dataclasses.dataclass
class A:
    name: str

@dataclasses.dataclass
class B(A):
    items: list[A | B]

data = [
    A('a'), 
    B('b', [A('c'), 
            B('d', [A('e'), A('f'), A('g'), A('h')]), 
            A('i'), A('j'), A('k'), A('l'), A('m'), A('n'), A('o')]),
    B('p', [A('q'), A('r'), A('s'), A('t'), A('u'), A('v'), A('w'), A('x'), A('y'), A('z')])
]
# tree = AnyTreeWidget(data)

# height = 220
# dcons = DebugWidget(show=False, height=height-4)
# tree.observe(lambda x: dcons.log(str(tree.value)), names='selected_nodes')  # type: ignore

# select_detail_app(tree, dcons.form, height=height)
app = TreeSelect(data, pane_widths=['130px', 4, 4])


Box(children=(SelectDetailApp_CSS(), ReflectWidget(), TreeSelect(children=(AnyTreeWidget(animation=100, layout…

## FilesWidget

In [ ]:
#| exporti
class FileNode(ValueNode):
    _icon_: ClassVar[str] = 'file'

class FilesTreeAdapter(TreeAdapter):
    @classmethod
    def is_folder(cls, x: Path) -> bool: return not x.is_file()
    @classmethod
    def items(cls, value):
        pp = [(p.name, p) for p in value.iterdir() if not p.name.startswith('.')]
        pp = sorted(pp, key=lambda x: x[0])
        for p in pp:
            yield p


In [ ]:
#| export
class FilesWidget(AnyTreeWidget):
    _adapter_ = FilesTreeAdapter()
    _value_node_ = FileNode


In [ ]:
# tree = FilesWidget([Path(pre + 'tests/images/other')])

height = 420
# dcons = DebugWidget(show=False, height=height-4)
# tree.observe(lambda x: dcons.log(str(tree.value)), names='selected_nodes')  # type: ignore

# select_detail_app(tree, dcons.form, height=height, widths=[1, 2, 1])
app = TreeSelect([Path(pre + 'tests/images/other')], tree_class=FilesWidget, 
                    expand=(0,),
                    dcons=True,
                    height=height, pane_widths=[1, 2, 1])
# app.tree.observe(lambda x: app.dcons.log(app.tree.path_of_node(x['new'][-1])), names='value')  # type: ignore
app.tree.observe(lambda x: app.dcons.log(app.tree.selected_paths), names='value')  # type: ignore
# app.tree.observe(lambda x: app.dcons.log(x['new']), names='value')  # type: ignore
test_eq(app.value, {})


Box(children=(SelectDetailApp_CSS(), ReflectWidget(), TreeSelect(children=(FilesWidget(animation=100, layout=L…

DebugWidget(children=(Box(children=(Button(description='clear', layout=Layout(width='2em'), style=ButtonStyle(…

In [ ]:
app.tree.value = ()


In [ ]:
nodes = app.tree.nodes_of_value(Path(pre + 'tests/images/other/grid.png'))
app.tree.value = nodes
test_eq(app.value, {'select': nodes})


In [ ]:
nodes = app.tree.nodes_of_value(Path(pre + 'tests/images/other/puppy/puppy-facemask8.tif'))
app.tree.select(Path(pre + 'tests/images/other/puppy/puppy-facemask8.tif'))
test_eq(app.value, {'select': nodes})


In [ ]:
node = app.tree.find_node(Path(pre + 'tests/images/other/puppy/puppy-multi.tif'))
test_eq(node.value, Path(pre + 'tests/images/other/puppy/puppy-multi.tif'))


In [ ]:
app.tree.source = [Path(pre + 'tests/images/strip')]


In [ ]:
app.tree.source = [Path(pre + 'tests/images/sqrt')]
test_eq(app.value, {'select': ()})
nodes = app.tree.nodes_of_value(Path(pre + 'tests/images/other/grid.png'))
app.tree.value = nodes
test_eq(app.value, {'select': ()})


## Colophon
 ----

In [ ]:
import fastcore.all as F
import nbdev; nbdev.nbdev_export('02_tree_widget.ipynb')